# Formatted zone

In [8]:
import os
import requests

import pandas as pd
import duckdb 


In [9]:
household_columns = ['section','delete','total','single_women_aged_16_to_64','single_men_aged_16_to_64','single_women_aged_65_or_over','single_men_aged_65_or_over',
    'adult_women_with_one_or_more_minors','adult_men_with_one_or_more_minors','two_adults_from_16_to_64_and_without_minors',
    'two_adults_one_at_least_65_and_without_minors','two_adults_and_one_minor','two_adults_and_two_minors','two_adults_and_three_or_more_minors',
    'two_adults_over_35_and_one_adult_from_16_to_34','two_adults_over_35_and_one_adult_from_16_to_34_and_one_minor',
    'two_adults_over_35_and_one_adult_from_16_to_34_and_two_minors','three_adults_and_0_or_more_minors','two_adults_over_35_and_two_adults_from_16_to_34',
    'two_adults_over_35_and_two_adults_from_16_to_34_and_one_minor','two_adults_over_35_and_two_adults_from_16_to_34_and_two_or_more_minors',
    'four_adults_and_0_or_more_minors','five_adults_and_0_or_more_minors','fifteen_or_more_inhabitants','only_minors']

In [10]:

def read_household(file):
    path = os.getcwd()+'/landing/persistent/'
    # Reads the Household Excel file
    # Returns the data in DataFrame format
    df = pd.read_excel(path+file,sheet_name='Composicion del hogar',header=[5],names=household_columns,na_values=None)
    df['section'].fillna(df['delete'],inplace=True)
    df.drop(labels='delete', axis=1, inplace=True)
    df.dropna(inplace = True)
    return df


def read_nationalities(file):
    path = os.getcwd()+'/landing/persistent/'
    # Reads the Nationalities Excel
    # Returns the data in DataFrame format
    df = pd.read_excel(path+file,sheet_name='Total',header=[7],na_values=None)
    df.rename(columns = {'Unnamed: 0':'Code','Unnamed: 1':'Madrid','Unnamed: 2':'Habitantes','Unnamed: 3':'Españoles','Unnamed: 4':'Extranjeros'}, inplace = True)
    df.drop(df.filter(regex="Unname"),axis=1, inplace=True)
    df.drop(df.filter(like='Total'),axis=1, inplace=True)
    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.replace(' ', '_')
    df.dropna(inplace = True)
    return df


def createTable(DB,df,table_name):
   # Creates a persistent table in DuckDB from the contents of the DataFrame
    con = duckdb.connect(DB)
    con.register(table_name,df)
    con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
    con.close()

In [11]:
# Formated _dod
def formatedZone():
    # current directory set to persistent folder
    persistent_dir = os.getcwd()+'/landing/persistent/'
    print(os.listdir(persistent_dir))
    for filename in os.listdir(persistent_dir):
        print(filename)
        
        if(filename.__contains__("household")):
            df = read_household(filename)
            display(df)
            table_name = filename.split('_')[1].split('.')[0]
            createTable('household.duckdb',df,table_name)

        if(filename.__contains__("nationalities")):
            df = read_nationalities(filename)
            display(df)
            table_name = filename.split('_')[1].split('.')[0]
            createTable('nationalities.duckdb',df,table_name)
            



In [12]:
formatedZone()


['2022-10-16_household2020.xls', '2022-10-16_household2018.xls', '2022-10-16_household2019.xls', '2022-10-16_nationalities2020.xls', '2022-10-16_nationalities2018.xls', '2022-10-16_nationalities2019.xls']
2022-10-16_household2020.xls


,section,total,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,...,two_adults_over_35_and_one_adult_from_16_to_34_and_one_minor,two_adults_over_35_and_one_adult_from_16_to_34_and_two_minors,three_adults_and_0_or_more_minors,two_adults_over_35_and_two_adults_from_16_to_34,two_adults_over_35_and_two_adults_from_16_to_34_and_one_minor,two_adults_over_35_and_two_adults_from_16_to_34_and_two_or_more_minors,four_adults_and_0_or_more_minors,five_adults_and_0_or_more_minors,fifteen_or_more_inhabitants,only_minors
1,Ciudad de Madrid,1307682.0,117998.0,116863.0,126695.0,37538.0,26232.0,5328.0,163183.0,175581.0,...,25255.0,9283.0,106779.0,51993.0,8131.0,3537.0,48740.0,59406.0,622.0,422.0
3,01. Centro,69187.0,11695.0,13596.0,5057.0,2269.0,815.0,172.0,13563.0,5267.0,...,476.0,213.0,4719.0,1123.0,181.0,67.0,1898.0,2153.0,48.0,25.0
4,1001.0,568.0,75.0,93.0,52.0,17.0,6.0,5.0,97.0,74.0,...,3.0,2.0,36.0,11.0,3.0,0.0,20.0,20.0,0.0,0.0
5,1002.0,492.0,111.0,99.0,32.0,17.0,4.0,1.0,96.0,30.0,...,1.0,0.0,33.0,6.0,1.0,0.0,15.0,14.0,0.0,0.0
6,1003.0,862.0,137.0,164.0,68.0,43.0,7.0,2.0,148.0,80.0,...,10.0,3.0,50.0,16.0,7.0,0.0,28.0,37.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482,21029.0,514.0,29.0,26.0,6.0,7.0,17.0,7.0,41.0,23.0,...,30.0,10.0,25.0,49.0,10.0,0.0,14.0,18.0,0.0,0.0
2483,21030.0,561.0,35.0,59.0,27.0,8.0,17.0,5.0,61.0,50.0,...,19.0,4.0,44.0,43.0,5.0,0.0,13.0,17.0,0.0,0.0
2484,21031.0,988.0,106.0,96.0,17.0,9.0,37.0,7.0,156.0,40.0,...,24.0,7.0,33.0,32.0,7.0,1.0,15.0,28.0,1.0,0.0
2485,21032.0,683.0,71.0,65.0,16.0,8.0,31.0,17.0,105.0,27.0,...,24.0,7.0,23.0,30.0,6.0,0.0,8.0,7.0,1.0,0.0


2022-10-16_household2018.xls


,section,total,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,...,two_adults_over_35_and_one_adult_from_16_to_34_and_one_minor,two_adults_over_35_and_one_adult_from_16_to_34_and_two_minors,three_adults_and_0_or_more_minors,two_adults_over_35_and_two_adults_from_16_to_34,two_adults_over_35_and_two_adults_from_16_to_34_and_one_minor,two_adults_over_35_and_two_adults_from_16_to_34_and_two_or_more_minors,four_adults_and_0_or_more_minors,five_adults_and_0_or_more_minors,fifteen_or_more_inhabitants,only_minors
1,Ciudad de Madrid,1278258.0,116921.0,114806.0,124815.0,35963.0,27460.0,5488.0,156118.0,173809.0,...,25188.0,8792.0,102020.0,50724.0,7803.0,3361.0,45696.0,50463.0,513.0,342.0
3,01. Centro,66411.0,11695.0,13210.0,5173.0,2142.0,970.0,191.0,12277.0,5164.0,...,453.0,191.0,4013.0,1060.0,183.0,79.0,1718.0,1854.0,43.0,18.0
4,1001.0,542.0,72.0,86.0,63.0,16.0,10.0,4.0,82.0,63.0,...,4.0,0.0,30.0,13.0,2.0,2.0,13.0,18.0,0.0,0.0
5,1002.0,481.0,100.0,91.0,36.0,19.0,7.0,0.0,89.0,28.0,...,2.0,1.0,38.0,9.0,1.0,1.0,10.0,11.0,0.0,1.0
6,1003.0,884.0,154.0,179.0,74.0,40.0,13.0,2.0,141.0,77.0,...,10.0,3.0,54.0,18.0,4.0,0.0,23.0,30.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482,21029.0,519.0,32.0,29.0,6.0,5.0,20.0,12.0,45.0,19.0,...,27.0,10.0,29.0,49.0,7.0,2.0,8.0,12.0,0.0,0.0
2483,21030.0,566.0,37.0,46.0,29.0,11.0,19.0,6.0,68.0,52.0,...,26.0,3.0,39.0,41.0,3.0,0.0,14.0,16.0,0.0,0.0
2484,21031.0,887.0,88.0,91.0,15.0,6.0,39.0,9.0,156.0,31.0,...,24.0,6.0,28.0,21.0,6.0,2.0,10.0,16.0,0.0,0.0
2485,21032.0,627.0,65.0,71.0,17.0,4.0,27.0,13.0,82.0,23.0,...,18.0,2.0,21.0,30.0,5.0,0.0,7.0,6.0,1.0,1.0


2022-10-16_household2019.xls


...

,Code,Madrid,Habitantes,Españoles,Extranjeros,Alemania,Austria,Bélgica,Bulgaria,Chipre,...,Uzbekistán,Vietnam,Yemen,Otros_Países_de_Asia,Australia,Fiji,Nueva_Zelanda,Timor_Oriental,Vanuatu,Otros_Países_de_Oceanía
3,0014,Acebeda (La),68.0,67.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0029,Ajalvir,4721.0,4073.0,648.0,12.0,0.0,1.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0035,Alameda del Valle,204.0,184.0,20.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0040,Álamo (El),9862.0,8372.0,1490.0,1.0,0.0,0.0,11.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0053,Alcalá de Henares,197562.0,165420.0,32142.0,110.0,13.0,13.0,1308.0,0.0,...,3.0,13.0,11.0,2.0,7.0,0.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,07,Nordeste Comunidad,66668.0,57421.0,9247.0,55.0,2.0,18.0,496.0,0.0,...,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
354,08,Sudeste Comunidad,108813.0,95507.0,13306.0,67.0,5.0,23.0,244.0,0.0,...,0.0,0.0,0.0,4.0,4.0,0.0,1.0,0.0,0.0,0.0
355,09,Sudoeste Comunidad,146669.0,132090.0,14579.0,59.0,7.0,22.0,331.0,1.0,...,0.0,3.0,0.0,2.0,4.0,0.0,1.0,0.0,0.0,0.0
356,10,Sierra Sur,36642.0,30884.0,5758.0,35.0,1.0,10.0,113.0,0.0,...,0.0,0.0,0.0,4.0,1.0,0.0,2.0,0.0,0.0,0.0


2022-10-16_nationalities2018.xls


,Code,Madrid,Habitantes,Españoles,Extranjeros,Alemania,Austria,Bélgica,Bulgaria,Chipre,...,Vietnam,Yemen,Otros_Países_de_Asia,Australia,Fiji,Nueva_Zelanda,Papúa_Nueva_Guinea,Timor_Oriental,Vanuatu,Otros_Países_de_Oceanía
3,0014,Acebeda (La),65.0,64.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0029,Ajalvir,4559.0,3953.0,606.0,8.0,0.0,1.0,13.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0035,Alameda del Valle,195.0,192.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0040,Alamo (El),9353.0,8143.0,1210.0,1.0,0.0,1.0,12.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0053,Alcalá de Henares,193751.0,163933.0,29818.0,89.0,12.0,13.0,1351.0,0.0,...,25.0,10.0,1.0,2.0,0.0,4.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,07,Nordeste Comunidad,63572.0,55648.0,7924.0,53.0,3.0,19.0,461.0,0.0,...,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0
354,08,Sudeste Comunidad,104366.0,92803.0,11563.0,65.0,3.0,23.0,233.0,0.0,...,0.0,0.0,6.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0
355,09,Sudoeste Comunidad,138537.0,126136.0,12401.0,57.0,7.0,19.0,327.0,0.0,...,2.0,0.0,2.0,7.0,0.0,1.0,1.0,0.0,0.0,0.0
356,10,Sierra Sur,34970.0,29932.0,5038.0,28.0,2.0,8.0,97.0,0.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


2022-10-16_nationalities2019.xls


,Code,Madrid,Habitantes,Españoles,Extranjeros,Alemania,Austria,Bélgica,Bulgaria,Chipre,...,Vietnam,Yemen,Otros_Países_de_Asia,Australia,Fiji,Nueva_Zelanda,Papúa_Nueva_Guinea,Samoa,Timor_Oriental,Otros_Países_de_Oceanía
3,0014,Acebeda (La),84.0,83.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0029,Ajalvir,4712.0,4098.0,614.0,8.0,0.0,1.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0035,Alameda del Valle,200.0,192.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0040,Alamo (El),9470.0,8165.0,1305.0,0.0,0.0,1.0,12.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0053,Alcalá de Henares,195649.0,164502.0,31147.0,100.0,14.0,15.0,1364.0,0.0,...,24.0,12.0,2.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,07,Nordeste Comunidad,65112.0,56644.0,8468.0,59.0,2.0,16.0,491.0,0.0,...,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
354,08,Sudeste Comunidad,106403.0,94076.0,12327.0,62.0,4.0,23.0,224.0,0.0,...,0.0,0.0,3.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0
355,09,Sudoeste Comunidad,142587.0,129153.0,13434.0,59.0,7.0,20.0,332.0,0.0,...,4.0,0.0,3.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0
356,10,Sierra Sur,35773.0,30471.0,5302.0,35.0,1.0,10.0,106.0,0.0,...,0.0,0.0,4.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0


## Verify tables created

In [13]:
# check if all tables in household.duckdb are created
con = duckdb.connect('household.duckdb')
household_tables = con.execute(f'SELECT table_name FROM information_schema.tables').df()
con.close()
household_tables


,table_name
0,household2018
1,household2020
2,household2019


In [14]:
# check if all tables in nationalities.db are created
con = duckdb.connect('nationalities.duckdb')
nationalities_tables = con.execute(f'SELECT table_name FROM information_schema.tables').df()
con.close()
nationalities_tables


,table_name
0,nationalities2018
1,nationalities2020
2,nationalities2019
